In [1]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
import random
import numpy as np

In [32]:
# def generate_individual(num_tasks, num_robots):
#     tasks = list(range(num_tasks))
#     random.shuffle(tasks)

#     # Evenly distribute tasks (some robots may have 1 more/less)
#     splits = np.array_split(tasks, num_robots)
#     return [list(s) for s in splits]


In [3]:
for _ in range(5):  # Generate 5 examples
    print(generate_individual(num_tasks=20, num_robots=3))    


[[2, 5, 15, 19, 9, 16, 0], [8, 10, 17, 18, 3, 12, 11], [13, 14, 6, 4, 7, 1]]
[[13, 12, 18, 9, 4, 16, 1], [5, 6, 3, 7, 17, 0, 15], [2, 10, 19, 8, 14, 11]]
[[7, 10, 2, 12, 15, 9, 3], [11, 0, 5, 6, 16, 1, 4], [14, 18, 17, 19, 13, 8]]
[[4, 3, 17, 8, 7, 6, 2], [9, 1, 16, 14, 13, 10, 15], [11, 19, 18, 12, 5, 0]]
[[7, 11, 15, 0, 10, 18, 4], [2, 1, 3, 6, 8, 19, 16], [5, 13, 14, 12, 9, 17]]


In [33]:
def generate_individual(num_tasks, num_robots):
    tasks = list(range(num_tasks))
    random.shuffle(tasks)

    # Generate random split sizes that sum to num_tasks
    split_sizes = [0] * num_robots
    for _ in range(num_tasks):
        split_sizes[random.randint(0, num_robots - 1)] += 1

    # Distribute tasks accordingly
    individual = []
    index = 0
    for size in split_sizes:
        individual.append(tasks[index:index+size])
        index += size
    # Ensure all tasks are assigned
    assert sum(split_sizes) == num_tasks, "Not all tasks assigned!"
    # print(individual)
    return individual

In [6]:
for _ in range(5):  # Generate 5 examples
    print(generate_individual(num_tasks=20, num_robots=3))    

[[1, 8, 0, 16, 17, 18, 14, 9, 19, 7], [11, 4, 6, 15, 10, 5], [2, 13, 12, 3]]
[[0, 4, 11, 10, 15, 2, 3], [6, 8, 13, 16, 7, 9, 12], [5, 17, 1, 19, 18, 14]]
[[9, 3, 13, 15, 19, 5, 4], [7, 10, 2, 12, 11, 17, 1], [0, 16, 6, 14, 18, 8]]
[[2, 9, 13, 14, 11, 16, 1, 3, 15], [19, 12, 17, 5, 18], [8, 10, 6, 7, 0, 4]]
[[19, 10, 8, 3, 13, 17, 14, 1, 12, 11], [0, 9, 7, 2], [18, 15, 6, 4, 5, 16]]


In [6]:
import random

def flatten(individual):
    return [task for robot in individual for task in robot]

def unflatten(flat, num_robots):
    """Random uneven split of flat list into num_robots sublists"""
    splits = [0] * num_robots
    for _ in flat:
        splits[random.randint(0, num_robots - 1)] += 1

    robot_seq, idx = [], 0
    for count in splits:
        robot_seq.append(flat[idx:idx+count])
        idx += count
    return robot_seq

def apply_order_shift(base_seq, diff_seq, F=0.5):
    """
    base_seq: original task order (list of task IDs)
    diff_seq: task order difference (list of task IDs)
    F: float between 0 and 1
    """
    new_seq = base_seq[:]
    task_indices = {task: i for i, task in enumerate(new_seq)}

    for i in range(len(new_seq)):
        if random.random() < F and new_seq[i] != diff_seq[i]:
            t1 = new_seq[i]
            t2 = diff_seq[i]
            j = task_indices[t2]
            # Swap t1 and t2 in new_seq
            new_seq[i], new_seq[j] = new_seq[j], new_seq[i]
            # Update mapping
            task_indices[t1], task_indices[t2] = task_indices[t2], task_indices[t1]
    return new_seq

def de_generate_offspring(x1, x2, x3, num_robots, F=0.5):
    """
    Differential evolution-inspired permutation operator
    """
    flat1 = flatten(x1)
    flat2 = flatten(x2)
    flat3 = flatten(x3)

    assert set(flat1) == set(flat2) == set(flat3), "Inconsistent task sets"

    # Difference in order
    diff = [t2 for t2 in flat2 if t2 in flat3]
    print("Difference in order:", diff)

    # Recombine
    mutant = apply_order_shift(flat1, diff, F=F)

    # Redistribute unevenly
    return unflatten(mutant, num_robots)


In [7]:
parent1 = generate_individual(num_tasks=20, num_robots=3)
parent2 = generate_individual(num_tasks=20, num_robots=3)
parent3 = generate_individual(num_tasks=20, num_robots=3)
# 3 parents selected from population
child = de_generate_offspring(parent1, parent2, parent3, num_robots=3, F=0.7)

print("Parent 1:", parent1)
print("Parent 2:", parent2)
print("Parent 3:", parent3)
print("Child:", child)


Difference in order: [2, 7, 17, 13, 12, 1, 0, 8, 6, 10, 19, 5, 4, 18, 11, 15, 3, 14, 16, 9]
Parent 1: [[9, 11, 10, 12, 13, 0, 6], [4, 1, 3, 17, 2, 18, 15], [19, 5, 7, 8, 14, 16]]
Parent 2: [[2, 7, 17, 13, 12, 1, 0], [8, 6, 10, 19, 5, 4, 18], [11, 15, 3, 14, 16, 9]]
Parent 3: [[12, 13, 1, 2, 18, 4, 10], [16, 14, 5, 6, 7, 17, 8], [11, 19, 9, 3, 15, 0]]
Child: [[14, 19, 17, 12, 13, 1, 0, 8, 6, 10], [7, 5, 4, 18, 11], [2, 3, 15, 16, 9]]


In [3]:
# Random positions for other tasks
TASK_COORDINATES = {}
for tid in range(0, 30):
    x = round(random.uniform(0, 1), 5)
    y = round(random.uniform(0, 1), 5)
    TASK_COORDINATES[tid] = (x, y)
#print the TASK_COORDINATES
print(f"Generated TASK_COORDINATES: {TASK_COORDINATES}")

Generated TASK_COORDINATES: {0: (0.69705, 0.65912), 1: (0.72966, 0.74986), 2: (0.1696, 0.0068), 3: (0.84208, 0.32632), 4: (0.83636, 0.36766), 5: (0.89403, 0.81913), 6: (0.10039, 0.20462), 7: (0.54908, 0.06478), 8: (0.13489, 0.9508), 9: (0.06335, 0.23856), 10: (0.79052, 0.4825), 11: (0.37287, 0.58046), 12: (0.78679, 0.00468), 13: (0.85591, 0.9104), 14: (0.03169, 0.23075), 15: (0.16304, 0.64752), 16: (0.56675, 0.21116), 17: (0.09113, 0.81652), 18: (0.52155, 0.74205), 19: (0.39718, 0.93534), 20: (0.12136, 0.68844), 21: (0.60087, 0.20341), 22: (0.85873, 0.20238), 23: (0.56802, 0.30087), 24: (0.34087, 0.24421), 25: (0.69079, 0.22627), 26: (0.77693, 0.21814), 27: (0.23676, 0.75051), 28: (0.57872, 0.44212), 29: (0.8314, 0.28604)}


In [80]:
import numpy as np
import random

def de_generate_offspring_allocation(x_i, x_r1, x_r2, x_r3, F=0.5, CR=0.7, num_robots=3):
    x_i = np.array(x_i)
    x_r1 = np.array(x_r1)
    x_r2 = np.array(x_r2)
    x_r3 = np.array(x_r3)

    print("x_i:", x_i)
    print("x_r1:", x_r1)    
    print("x_r2:", x_r2)
    print("x_r3:", x_r3)
    # Mutation in robot ID space (values between 0 and num_robots - 1)
    diff = x_r2 - x_r3
    mutant = x_r1 + F * diff

    # Add small noise to reduce bias, then floor and clamp
    mutant += np.random.uniform(-0.3, 0.3, size=mutant.shape)
    mutant = np.floor(mutant).astype(int)
    mutant = np.clip(mutant, 0, num_robots - 1)

    u_i = []
    j_rand = random.randint(0, len(x_i) - 1)

    for j in range(len(x_i)):
        r = random.random()
        if r < CR or j == j_rand:
            u_i.append(mutant[j])
        else:
            u_i.append(x_i[j])

    return u_i

def allocation_vector_to_robot_tasks(allocation_vector, num_robots):
    task_allocation = [[] for _ in range(num_robots)]
    for task_id, robot_id in enumerate(allocation_vector):
        robot_id = min(max(int(robot_id), 0), num_robots - 1)  # Ensure valid index
        task_allocation[robot_id].append(task_id)
    return task_allocation

def flatten(individual):
    return [task for robot in individual for task in robot]

def generate_offspring(parent, exemplar, mutation_prob=0.5, use_de=True, r1=None, r2=None, r3=None, F=0.5, CR=0.7):
    """
    Args:
        parent, exemplar: both are list-of-lists task sequences (robot-wise)
        use_de: whether to apply DE operator
        r1, r2, r3: optional DE parents (in list-of-lists form)

    Returns:
        new_individual: mutated child
    """
    num_robots = len(parent)
    tasks = set(t for r in parent for t in r)

    if use_de and r1 and r2 and r3:
        # Convert to allocation vector representation
        def to_allocation_vector(ind):
            vec = [0] * sum(len(r) for r in ind)
            for r_id, task_list in enumerate(ind):
                for t in task_list:
                    vec[t] = r_id
            return vec

        x_i = to_allocation_vector(parent)
        x_r1 = to_allocation_vector(r1)
        x_r2 = to_allocation_vector(r2)
        x_r3 = to_allocation_vector(r3)

        allocation_vec = de_generate_offspring_allocation(x_i, x_r1, x_r2, x_r3, F=F, CR=CR, num_robots=num_robots)
        allocation_vec = [min(max(int(rid), 0), num_robots - 1) for rid in allocation_vec]
        return allocation_vector_to_robot_tasks(allocation_vec, num_robots)

    # Otherwise use original exemplar-based mutation
    ex_flat = [t for r in exemplar for t in r if t in tasks]
    new_seq = ex_flat.copy()

    if random.random() < mutation_prob:
        for _ in range(3):
            i, j = random.sample(range(len(new_seq)), 2)
            new_seq[i], new_seq[j] = new_seq[j], new_seq[i]

    splits = [0] * num_robots
    for _ in new_seq:
        splits[random.randint(0, num_robots - 1)] += 1

    robot_seq, idx = [], 0
    for s in splits:
        robot_seq.append(new_seq[idx:idx + s])
        idx += s

    return robot_seq


In [84]:
# example usage
parent = generate_individual(num_tasks=20, num_robots=4)
exemplar = generate_individual(num_tasks=20, num_robots=4)
r1 = generate_individual(num_tasks=20, num_robots=4)
child = generate_offspring(parent, exemplar, use_de=True, r1=r1, r2=exemplar, r3=parent, F=0.7)
print("Parent:", parent)
print("Exemplar:", exemplar)
print("Child:", child)

x_i: [0 2 3 1 3 1 2 2 0 3 2 2 3 1 3 2 1 1 0 1]
x_r1: [0 0 2 3 1 3 2 3 0 2 1 3 3 3 2 2 1 2 3 3]
x_r2: [3 0 2 0 3 3 3 3 2 3 3 2 0 3 0 3 2 0 1 2]
x_r3: [0 2 3 1 3 1 2 2 0 3 2 2 3 1 3 2 1 1 0 1]
Parent: [[8, 18, 0], [17, 19, 13, 5, 3, 16], [15, 1, 10, 11, 6, 7], [9, 2, 4, 12, 14]]
Exemplar: [[3, 1, 12, 14, 17], [18], [8, 11, 2, 19, 16], [10, 4, 9, 6, 0, 15, 5, 13, 7]]
Child: [[0, 1, 8, 12, 14], [2, 5, 10, 16, 17], [3, 6, 9, 15], [4, 7, 11, 13, 18, 19]]
